In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [ ]:
from utils import fig_ax, load_data, save_results

In [ ]:
df_train_val, s_train_val, df_test = load_data()

In [ ]:
# send everything to numpy arrays
X_train_val = df_train_val.to_numpy()
y_train = s_train_val.to_numpy()
X_test = df_test.to_numpy()

In [ ]:
# compute predictions:
y_pred = np.random.randn(df_test.shape[0])

In [ ]:
save_results(y_pred, df_test.index)